In [1]:
import pandas as pd

In [2]:
# Requires the HPRC HiFi QC WDL to be run and then the hprc_intermediate_assembly/hpc/update_table_with_outputs.py script to create the hifi_samples_output_table.csv
output_df = pd.read_csv('hifi_qc/hifi_samples_output_table.csv')
output_df['aggregate-sample_id'] = [sample.split('_')[0] for sample in output_df['sample_id'].tolist()]

In [9]:
# Compare HPRC HiFi QC theoretical coverage to HPRC Pangenome Y4 Working Group expected coverage
pangenome_df = pd.read_csv('/private/groups/hprc/human-pangenomics/HPRC_metadata/data/production/Pangenome_HiFi_Libraries_Year4 - Totals_by_Sample.csv')

In [10]:
# Create a the post_sra_metadata qc data dictionary and flag low coverage samples
output_dict = {sample:[] for sample in output_df['aggregate-sample_id'].tolist()}
for data in output_df[['aggregate-sample_id','hifi_qc_wf.hifi_qc_summary']].values:
	output_dict[data[0]].append(pd.read_csv(data[1],sep='\t'))

for k,v in output_dict.items():
	output_dict[k] = pd.concat(v)
	output_dict[k]['theoretical_coverage'] = [data/3.1 for data in output_dict[k]['total_Gbp'].tolist()]
	theoretical_coverage = output_dict[k]['theoretical_coverage'].sum()
	assert round(theoretical_coverage,1) == pangenome_df[pangenome_df['Sample ID'].isin([k])]['Coverage'].tolist()[0]
	if theoretical_coverage < 60:
		print(k, theoretical_coverage)

readstats = pd.concat(output_dict.values())
readstats['sample_ID'] = [sample.split('_')[0] for sample in readstats['filename'].tolist()]
readstats.to_csv('../1_metadata/WUSTL_HPRC_HiFi_Year4_post_sra_metadata.csv',index=False)

NA20809 59.17096774193548
